In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'13edzdm9'

In [3]:
wandb.init(id='xjnaa652', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 54 s, sys: 2.16 s, total: 56.2 s
Wall time: 56.3 s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [9]:
"""for n, p in m.named_parameters():
    if 'fcs_2' in n:
        p.requires_grad = True
    else:
        p.requires_grad = False
        
    print(n, p.requires_grad)"""

"for n, p in m.named_parameters():\n    if 'fcs_2' in n:\n        p.requires_grad = True\n    else:\n        p.requires_grad = False\n        \n    print(n, p.requires_grad)"

In [10]:
model_stem = "9.17"
START_E = 46

In [11]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    opt.param_groups[0]['lr'] = 1e-4 #if e < 10 else 1e-4

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=1280)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 46

{'trn_control_loss': 0.00905067, 'trn_headings_loss': 0.00049345, 'trn_curvatures_loss': 0.00163623, 'consistency losses/trn_steer_cost': 0.00417531, 'consistency losses/trn_te_loss': 21.45942955, 'aux losses/trn_pitch_loss': 0.10203552, 'aux losses/trn_yaw_loss': 0.09688926, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.7109375, 'consistency losses/trn_torque_loss': 2786.29459369, 'consistency losses/trn_torque_delta_loss': 13.53946481, 'logistical/obs_consumed_per_second': 238.40944882, 'logistical/obs_generated_per_second': 168.92913386, 'logistical/data_consumption_ratio': 1.41026758, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.29476357, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 235.3046875, 'logistical/obs_generated_per_second': 170.640625, 'logistical/data_consumption_ratio': 1.3815032, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00608322, 'trn_headings_loss': 0.00028802, 'trn_curvatures_loss': 0.000

{'logistical/obs_consumed_per_second': 233.6796875, 'logistical/obs_generated_per_second': 175.078125, 'logistical/data_consumption_ratio': 1.3341144, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00713579, 'trn_headings_loss': 0.00043269, 'trn_curvatures_loss': 0.00128423, 'consistency losses/trn_steer_cost': 0.00315731, 'consistency losses/trn_te_loss': 21.98685995, 'aux losses/trn_pitch_loss': 0.09519053, 'aux losses/trn_yaw_loss': 0.08161151, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.7421875, 'consistency losses/trn_torque_loss': 3088.54561329, 'consistency losses/trn_torque_delta_loss': 13.04194252, 'logistical/max_param': 16.29602242, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 239.015625, 'logistical/obs_generated_per_second': 168.6953125, 'logistical/data_consumption_ratio': 1.41826795, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00720555, 'trn_headings_loss': 0.00035564, 'trn_curvatures_loss': 0.00124872, 'consi

{'logistical/obs_consumed_per_second': 237.140625, 'logistical/obs_generated_per_second': 170.8828125, 'logistical/data_consumption_ratio': 1.38796642, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01233104, 'trn_headings_loss': 0.00061675, 'trn_curvatures_loss': 0.00198601, 'consistency losses/trn_steer_cost': 0.00503448, 'consistency losses/trn_te_loss': 20.77391756, 'aux losses/trn_pitch_loss': 0.11727452, 'aux losses/trn_yaw_loss': 0.11430526, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.7890625, 'consistency losses/trn_torque_loss': 2842.15582466, 'consistency losses/trn_torque_delta_loss': 14.75528692, 'logistical/max_param': 16.2883606, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 237.46875, 'logistical/obs_generated_per_second': 172.5078125, 'logistical/data_consumption_ratio': 1.37719862, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00801904, 'trn_headings_loss': 0.00038489, 'trn_curvatures_loss': 0.00167113, 'consis

{'logistical/obs_consumed_per_second': 240.5625, 'logistical/obs_generated_per_second': 168.5390625, 'logistical/data_consumption_ratio': 1.42766024, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00580981, 'trn_headings_loss': 0.00029753, 'trn_curvatures_loss': 0.00096521, 'consistency losses/trn_steer_cost': 0.00331774, 'consistency losses/trn_te_loss': 22.37980618, 'aux losses/trn_pitch_loss': 0.08933425, 'aux losses/trn_yaw_loss': 0.09410381, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.6796875, 'consistency losses/trn_torque_loss': 2932.42023087, 'consistency losses/trn_torque_delta_loss': 10.7907428, 'logistical/max_param': 16.29036331, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 236.5, 'logistical/obs_generated_per_second': 169.15625, 'logistical/data_consumption_ratio': 1.39968562, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00556837, 'trn_headings_loss': 0.00028408, 'trn_curvatures_loss': 0.00075995, 'consistency lo

{'logistical/obs_consumed_per_second': 234.0703125, 'logistical/obs_generated_per_second': 171.1328125, 'logistical/data_consumption_ratio': 1.36816904, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00820418, 'trn_headings_loss': 0.00035526, 'trn_curvatures_loss': 0.00122909, 'consistency losses/trn_steer_cost': 0.00454363, 'consistency losses/trn_te_loss': 19.81702295, 'aux losses/trn_pitch_loss': 0.07736206, 'aux losses/trn_yaw_loss': 0.09885669, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.75, 'consistency losses/trn_torque_loss': 2613.75061512, 'consistency losses/trn_torque_delta_loss': 13.73904522, 'logistical/max_param': 16.28312111, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 234.0859375, 'logistical/obs_generated_per_second': 169.3984375, 'logistical/data_consumption_ratio': 1.38236925, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0077537, 'trn_headings_loss': 0.00041207, 'trn_curvatures_loss': 0.0011562, 'consisten

...

{'logistical/obs_consumed_per_second': 237.921875, 'logistical/obs_generated_per_second': 170.2265625, 'logistical/data_consumption_ratio': 1.39845048, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0082725, 'trn_headings_loss': 0.00048121, 'trn_curvatures_loss': 0.00135548, 'consistency losses/trn_steer_cost': 0.00373951, 'consistency losses/trn_te_loss': 21.25571217, 'aux losses/trn_pitch_loss': 0.09348607, 'aux losses/trn_yaw_loss': 0.09409237, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.703125, 'consistency losses/trn_torque_loss': 2895.56286049, 'consistency losses/trn_torque_delta_loss': 10.91667834, 'logistical/max_param': 16.28161812, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 234.6484375, 'logistical/obs_generated_per_second': 169.7007874, 'logistical/data_consumption_ratio': 1.38105598, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.0074089, 'trn_headings_loss': 0.00042916, 'trn_curvatures_loss': 0.00112106, 'consis

{'logistical/obs_consumed_per_second': 236.71875, 'logistical/obs_generated_per_second': 169.1875, 'logistical/data_consumption_ratio': 1.3992166, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00922091, 'trn_headings_loss': 0.00041695, 'trn_curvatures_loss': 0.00147547, 'consistency losses/trn_steer_cost': 0.00445563, 'consistency losses/trn_te_loss': 21.56747905, 'aux losses/trn_pitch_loss': 0.14611959, 'aux losses/trn_yaw_loss': 0.08940959, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.796875, 'consistency losses/trn_torque_loss': 2963.08985233, 'consistency losses/trn_torque_delta_loss': 12.44211139, 'logistical/max_param': 16.28214836, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 236.5859375, 'logistical/obs_generated_per_second': 168.015625, 'logistical/data_consumption_ratio': 1.40806805, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00890964, 'trn_headings_loss': 0.0004253, 'trn_curvatures_loss': 0.00156075, 'consistency

{'logistical/obs_consumed_per_second': 176.0, 'logistical/obs_generated_per_second': 173.0, 'logistical/data_consumption_ratio': 1.01589929, 'logistical/manual_train_pause': 0.0}


 Eval...
EPOCH 76

{'trn_control_loss': 0.00700751, 'trn_headings_loss': 0.00036146, 'trn_curvatures_loss': 0.0013105, 'consistency losses/trn_steer_cost': 0.00351342, 'consistency losses/trn_te_loss': 21.53584523, 'aux losses/trn_pitch_loss': 0.09730887, 'aux losses/trn_yaw_loss': 0.08216465, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.75, 'consistency losses/trn_torque_loss': 2957.13016939, 'consistency losses/trn_torque_delta_loss': 12.1241096, 'logistical/obs_consumed_per_second': 237.99212598, 'logistical/obs_generated_per_second': 172.63779528, 'logistical/data_consumption_ratio': 1.37937586, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.28147888, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 235.8828125, 'logistical/obs_generated_per_second': 169.9296

{'logistical/obs_consumed_per_second': 241.3671875, 'logistical/obs_generated_per_second': 170.3984375, 'logistical/data_consumption_ratio': 1.41636783, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00714098, 'trn_headings_loss': 0.00036827, 'trn_curvatures_loss': 0.00117959, 'consistency losses/trn_steer_cost': 0.0035436, 'consistency losses/trn_te_loss': 21.13822808, 'aux losses/trn_pitch_loss': 0.09185147, 'aux losses/trn_yaw_loss': 0.09770203, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.5, 'consistency losses/trn_torque_loss': 2574.859478, 'consistency losses/trn_torque_delta_loss': 10.79442732, 'logistical/max_param': 16.28252792, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 241.3046875, 'logistical/obs_generated_per_second': 169.4140625, 'logistical/data_consumption_ratio': 1.42492437, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00617178, 'trn_headings_loss': 0.0003151, 'trn_curvatures_loss': 0.00112196, 'consistency 

KeyboardInterrupt: 

In [12]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e77.torch")

In [13]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
